<a href="https://colab.research.google.com/github/munteanuic/RankALS/blob/main/Replication_Assignment_Ioana_Munteanu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import pandas as pd

In [ ]:
!pip install surprise

# Data Processing
I decided to use MovieLens 100K movie ratings [1]. MovieLens data sets were collected by the GroupLens Research Project
at the University of Minnesota.
 
This data set consists of:
  - 100,000 ratings (1-5) from 943 users on 1682 movies. 
  - Each user has rated at least 20 movies. 
  - Simple demographic info for the users (age, gender, occupation, zip)

In [ ]:
from surprise import Dataset
data = Dataset.load_builtin('ml-100k')

In [ ]:
'''Source: CSCI 5123 Fall 2021 Assignment 2'''
from surprise.model_selection import KFold
import copy
tmp = pd.DataFrame(data.raw_ratings, columns=["userId", "movieId", "rating", "tstamp"])
rating_df = tmp.pivot(index='userId', columns='movieId', values='rating')

def partition (list_in, n):
    random.shuffle(list_in)
    return [list_in[i::n] for i in range(n)]

non_rated_movies = []
for userId in rating_df.index:
  tmp = rating_df.loc[userId].loc[rating_df.loc[userId].isna()].index
  for movieId in tmp:
    non_rated_movies.append((userId, movieId, 0))
n_splits=5
non_rated_testset = partition(non_rated_movies, n_splits)

def my_split(data, n_splits=5, random_state=2021):
    """ 
    The method to generate the splits of trainset, testset_rated (testset with rated items only), testset_withunrated (testset with unrated items, and set their ''true rating'' as 0)
    Parameters: 
    data: suprise dataset, https://surprise.readthedocs.io/en/stable/dataset.html#surprise.dataset.Dataset
    n_splits: number of splits
    random_state: seed 
    Returns: 
    dataset: a list (length = n_splits) of tuples (trainset, testset_rated, testset_withunrated), 
            and each of the trainset/testset is a list of tuples (userId, movieId, true rating)
  
    """
    kf = KFold(n_splits=n_splits, random_state=random_state)

    dataset = []
    i = 0
    tmp = copy.deepcopy(data)
    for trainset, testset_rated in kf.split(tmp):
      testset_withunrated = copy.deepcopy(testset_rated)
      testset_withunrated.extend(non_rated_testset[i][:len(testset_rated)])
      dataset.append((trainset, testset_rated, testset_withunrated))
      i += 1
    return dataset


# Implementation of RankALS
I used the pseudocode in the paper "Alternating least squares for personalized ranking" [2]. It is described on page 87 as Algorithm 2. The paper proposes "an effective approach for the direct minimization of a ranking objective function, without sampling." It uses an algorithm similar to ImplicitALS, having the same time complexity.

The algorithm has tunable paramaters such as:
  - E: number of iterations (defaulted to 10)
  - rho: initialization range paramter (number between 0 and 1 randomly generated)
  - s: True if item importance weights in the ojective funtcion will be used and False otherwise
  - F: number of features (defaulted to 20)

The algorithm was mathematically derrived by the authors and then implmented using different statistics: p_bar, p_curl, A_bar, A_curl, p1_bar_bar, p2_bar_bar, p3_bar_bar. Every iteration, there is a P step (Q is constant) and a Q step (P is contant). In the P step, the user X feature matrix is updated and in the Q step, the item X feature matrix is updated.

##Helper functions

In [ ]:
'''
Paramters:
  matrix: a matrix
  S: a vector conaining row indices
Output: 
  a submatrix of matrix containg all the rows corresponding to the indices from S
'''
def submatrix_row(matrix, S):
    num_rows, num_cols = matrix.shape
    return(matrix[np.ix_(S,np.arange(num_cols))])

'''
Paramters:
  matrix: a matrix
  S1: a vector conaining row indices
  S2: a vector conaining column indices
Output: 
  a submatrix of matrix containg all the rows corresponding to the indices from S1 intersected with the columns corresponding to the indices from S2
'''
def submatrix_row_col(matrix, S1, S2):
    return(matrix[np.ix_(S1,S2)])

'''
Paramters:
  vector: a matrix
  S: a vector conaining row indices
Output: 
  a subvector of vector containg all the rows corresponding to the indices from S
  OR prints "Try transposing" if the vector is nx1 instead of 1xn
'''
def subvector(vector, S):
    if vector.ndim == 1:
        return np.take(vector, S)
    else:
        print("Try transposing")

def is_pos(x): 
  return x > 0

def is_max(x):
  return x == 5

In [ ]:
from numpy.core.numeric import False_
from surprise.prediction_algorithms.predictions import Prediction
class RankALS():
    def __init__(self, trainset, rho, s = False, E=10, F=20):
        self.E = E
        self.rho = rho
        self.F = F
        raw_ratings = pd.DataFrame([x for x in trainset.all_ratings()],columns=["userId", "movieId", "ratings"])
        self.ratings = raw_ratings.pivot(index='userId', columns='movieId', values='ratings')
        self.ratings.index = self.ratings.index.map(str)# get the list of userId (type str)
        self.ratings.columns = self.ratings.columns.map(str)# get the list of movieId (type str)
        self.predict_ratings = pd.DataFrame(index=self.ratings.index, columns=self.ratings.columns)
        self.R = self.ratings.to_numpy()
        if s == False:
          self.s = np.ones(self.R.shape[1])
        else:
          self.s = []
          for i in range(self.R.shape[1]):
            R_tr = np.transpose(self.R)
            all_rating_user = R_tr[i, :]
            bool_arr = is_pos(all_rating_user)
            self.s.append(np.where(bool_arr)[0].shape[0])
          self.s = np.array(self.s)

    def fit(self):
        num_users, num_items = self.R.shape
        l_curl = np.sum(self.s)
        P = np.random.uniform(self.rho * (-1), self.rho, (num_users, self.F))
        Q = np.random.uniform(self.rho * (-1), self.rho, (num_items, self.F))

        for e in range (self.E):
            print ("Iteration {}".format(e))
            #P-step 
            Q_tr = np.transpose(Q)
            q_curl = np.matmul (Q_tr, self.s)
            A_curl = np.matmul (np.matmul(Q_tr, np.diag(self.s)), Q)
            z = []

            for u in range(num_users):
                # calcualate r_u
                # https://www.kite.com/python/answers/how-to-find-the-index-of-list-elements-that-meet-a-condition-in-python
                all_rating_user = self.R[u, :]
                bool_arr = is_pos(all_rating_user)
                I_u = np.where(bool_arr)[0]
                r_u = np.transpose(submatrix_row_col(self.R, np.array([u]), I_u))
                
                # calcualate l_bar
                l_bar = I_u.shape[0] # get num_items from I_U
                z.append(l_bar) # calculate z that will be used in Q-step

                # calculate r_bar
                r_u_tr = np.transpose(r_u)
                r_bar = np.matmul(r_u_tr, np.ones(r_u_tr.shape[1]))
                
                # calculate q_bar
                Q_I_u = submatrix_row(Q, I_u)
                Q_I_u_tr = np.transpose(Q_I_u)
                q_bar = np.matmul(Q_I_u_tr, np.ones(Q_I_u_tr.shape[1]))

                # calculate b_bar
                b_bar = np.matmul(Q_I_u_tr, r_u)


                # calculate A_bar
                A_bar = np.matmul(Q_I_u_tr, Q_I_u)

                # calculate r_curl
                s_I_u = subvector(self.s, I_u)
                s_I_u_tr = np.transpose(s_I_u)
                r_curl = np.matmul(s_I_u_tr, r_u)

                # calculate b_curl
                diag = np.diag(s_I_u)
                b_curl = np.matmul(np.matmul(Q_I_u_tr, diag), r_u)
                t1 = l_curl * A_bar
                t2 = np.matmul(q_bar, np.transpose(q_curl))
                t3 = np.matmul(q_curl, np.transpose(q_bar)) 
                t4 = l_bar * A_curl
                M = t1 - t2 - t3 + t4
                # print("T1: {}, T2: {}, T3: {}, T4: {}".format(t1,t2,t3,t4))

                #calculate y
                t1 = np.transpose(b_bar * l_curl)
                t2 = (q_bar * r_curl)
                t3 = (r_bar * q_curl)
                t4 = np.transpose(l_bar * b_curl)
                y = t1 - t2 - t3 + t4
                y = np.transpose(y)

                # update P[u]
                P_u = np.matmul(np.linalg.inv(M), y)
                P[u] = np.transpose(P_u)

            #Q-step
            # calculate q_curl
            q_curl = np.matmul (Q_tr, self.s)

            # calculate A_bar_bar
            P_tr = np.transpose(P)
            z = np.array(z)
            A_bar_bar = np.matmul(np.matmul(P_tr, np.diag(z)), P)
            
            # calculate p_bar, never used
            p1_bar = np.matmul(P_tr, z)
            
            r_curl, r_bar, Q_bar = np.zeros(num_users), np.zeros(num_users), np.zeros((num_users, self.F))
            for u in range(num_users):
                # calcualate r_u
                all_rating_user = self.R[u, :]
                bool_arr = is_pos(all_rating_user)
                I_u = np.where(bool_arr)[0]
                r_u = np.transpose(submatrix_row_col(self.R, np.array([u]), I_u))

                # calculate r_u_curl
                r_u_tr = np.transpose(r_u)
                s_I_u = subvector(self.s, I_u)
                r_curl[u] = (np.matmul(r_u_tr, s_I_u))

                # calculate r_bar
                r_bar[u] = np.matmul(r_u_tr, np.ones(r_u_tr.shape[1]))

                # calculate Q_bar_u
                Q_I_u_tr = np.transpose(submatrix_row(Q, I_u))
                
                # calculate Q_bar
                Q_bar[u] = np.matmul(Q_I_u_tr, np.ones(Q_I_u_tr.shape[1]))
            
            # calculate p2_bar_bar
            P_tr = np.transpose(P)
            tr = np.trace(np.matmul(P_tr, Q_bar))
            p2_bar_bar = P_tr * tr
            p3_bar_bar = np.matmul(np.transpose(P), r_bar)

            for i in range(num_items):
                # calculate r_i
                R_tr = np.transpose(self.R)
                all_rating_user = R_tr[i, :]
                bool_arr = is_pos(all_rating_user)
                U_i = np.where(bool_arr)[0]
                r_i = np.transpose(submatrix_row_col(R_tr, np.array([i]), U_i))
            
                # calculate b_bar
                P_tr_U_i = np.transpose(submatrix_row(P, U_i))
                b_bar = np.matmul(P_tr_U_i, r_i)
                
                # calculate A_bar
                A_bar = np.matmul(P_tr_U_i, submatrix_row(P, U_i))

                # calculate A_bar_bar
                b_bar_bar = np.matmul(np.matmul(P_tr_U_i, np.diag(subvector(z, U_i))), r_i)

                # calculate p1_bar_bar
                p1_bar_bar = np.matmul(P_tr_U_i, subvector(r_curl, U_i))

                # calculate M
                M = l_curl * A_bar + self.s[i] * A_bar_bar

                # calculate y
                t1 = np.matmul(A_bar, q_curl)
                t2 = (np.transpose(b_bar * l_curl))[0]
                t3 = p1_bar_bar 
                t4 = p3_bar_bar*self.s[i]
                t5 = np.transpose(b_bar_bar*self.s[i])[0]
                y = t1 + t2 - t3 - t4 + t5
                q_i = np.matmul(np.linalg.inv(M), y)
                if q_i[0] == np.nan:
                  Q[i] = np.random.uniform(self.rho * (-1), self.rho, (self.F))
                else:
                  Q[i] = q_i

        pred = np.matmul(P, np.transpose(Q))
        pred[pred > 0] = 1
        pred[pred <= 0] = 0
        self.predict_ratings = pd.DataFrame(data = pred, index = self.ratings.index, columns = self.ratings.columns)
        
    def test(self, testset):
      estimations = []
      for userId, movieId, true_rating in testset:
        if userId in self.predict_ratings.index and movieId in self.predict_ratings.columns and not np.isnan(self.predict_ratings.loc[userId, movieId]):
          pred_rating = self.predict_ratings.loc[userId, movieId]
          estimations.append(Prediction(uid=userId, iid=movieId, r_ui=true_rating, est=pred_rating,details={'was_impossible': False}))
      return estimations

# Metrics and comparison to SVDpp
I calculated the error rate and the recall, just like in the paper. Additioanlly, I added three more metrics: precision, RMSE, and MAE. I tested the algorithm both with and without an importance weight. Addionally, I compared the two versions of the RankALS algorithm with SVDpp since they are both algorithms that use matrix factorization and implicit bias. Considering that the Movielens dataset is sparse, the implicit bias is expected to produce better results. 

In [ ]:
'''
Source: https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-compute-precision-k-and-recall-k
Used to calculate precision and recall
'''

from collections import defaultdict

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [ ]:
from surprise import accuracy, SVDpp
from surprise.model_selection import cross_validate

metric_report = pd.DataFrame(index=['RankALS without weight', 'RankALS with weight', 'SVD++'], columns=['error rate', 'precision', 'recall', 'rmse','mae'])
split_data = my_split(data)

errors0, precs0, recs0, rmses0, maes0 = [], [], [], [], []
errors1, precs1, recs1, rmses1, maes1 = [], [], [], [], []
errors2, precs2, recs2, rmses2, maes2 = [], [], [], [], []

for fold in split_data:
  trainset, testset_rated, testset_withunrated = fold
  testset_rated = [[x[0], x[1], 1] if x[2] == 5 else [x[0], x[1], 0] for x in testset_rated]
  rho = np.random.normal(0,1,1)

  print("-------Start testing Rank ALS without wighting------")
  als = RankALS(trainset, rho, E = 10) 
  als.fit()
  pred = als.test(testset_rated)

  # calculate error rate 
  right, wrong = 0, 0
  for p in pred:
    if p.est == p.r_ui:
      right += 1
    else:
      wrong += 1
  errors0.append(wrong/(right+wrong))

  # calculate precisions and recalls 
  precisions, recalls = precision_recall_at_k(pred, k=5, threshold=1) # set threshold to 1 since 1 means recommended
  precs0.append(sum(prec for prec in precisions.values()) / len(precisions))
  recs0.append(sum(rec for rec in recalls.values()) / len(recalls))

  # calculate rmses
  rmses0.append(accuracy.rmse(pred))

  # calculate maes
  maes0.append(accuracy.mae(pred))
  print("-------End testing Rank ALS without weighting------")

  print("-------Start testing Rank ALS with weighting------")
  alsW = RankALS(trainset, rho, s = True, E = 10) 
  alsW.fit()
  pred = alsW.test(testset_rated)

  # calculate error rate 
  right, wrong = 0, 0
  for p in pred:
    if p.est == p.r_ui:
      right += 1
    else:
      wrong += 1
  errors1.append(wrong/(right+wrong))

  # calculate precisions and recalls 
  precisions, recalls = precision_recall_at_k(pred, k=5, threshold=1) # set threshold to 1 since 1 means positive rating
  precs1.append(sum(prec for prec in precisions.values()) / len(precisions))
  recs1.append(sum(rec for rec in recalls.values()) / len(recalls))

  # calculate rmses
  rmses1.append(accuracy.rmse(pred))

  # calculate maes
  maes1.append(accuracy.mae(pred))
  print("-------End testing Rank ALS with weighting------")

  print("-------Start testing SVDpp------")
  trainset, testset_rated, testset_withunrated = fold
  svdpp = SVDpp(n_factors=20, n_epochs=10)
  svdpp.fit(trainset)
  pred = svdpp.test(testset_rated)

  # calculate error rate 
  right, wrong = 0, 0
  for p in pred:
    if p.est == p.r_ui:
      right += 1
    else:
      wrong += 1
  errors2.append(wrong/(right+wrong))

  # calculate precisions and recalls 
  precisions, recalls = precision_recall_at_k(pred, k=5, threshold=5) # set threshold to 5 since 5 means psoitive rating
  precs2.append(sum(prec for prec in precisions.values()) / len(precisions))
  recs2.append(sum(rec for rec in recalls.values()) / len(recalls))

  # calculate rmses
  rmses2.append(accuracy.rmse(pred))

  # calculate maes
  maes2.append(accuracy.mae(pred))
  
  print("-------End testing SVDpp------")
metric_report['error rate'] = [sum(errors0)/5, sum(errors1)/5, sum(errors2)/5]
metric_report['precision'] = [sum(precs0)/5, sum(precs1)/5, sum(precs2)/5]
metric_report['recall'] = [sum(recs0)/5, sum(recs1)/5, sum(recs2)/5]
metric_report['rmse'] = [sum(rmses0)/5, sum(rmses1)/5, sum(rmses2)/5]
metric_report['mae'] = [sum(maes0)/5, sum(maes1)/5, sum(maes2)/5]

print(metric_report)

-------Start testing Rank ALS without wighting------
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
RMSE: 0.5385
MAE:  0.2900
-------End testing Rank ALS without weighting------
-------Start testing Rank ALS with weighting------
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
RMSE: 0.5290
MAE:  0.2799
-------End testing Rank ALS with weighting------
-------Start testing SVDpp------
RMSE: 0.9155
MAE:  0.7271
-------End testing SVDpp------
-------Start testing Rank ALS without wighting------
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
RMSE: 0.5448
MAE:  0.2968
-------End testing Rank ALS without weighting------
-------Start testing Rank ALS with weighting------
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Ite

# Results
RankALS with weight performs slightly better than RankALS without weight in terms of eror rate, rmse and mae, while RankALS without weight slightly outperforms RankALS for recall and precision. The major differences are between both RankALS adn SVD++. The error rate forr SVD++ is 3 times higher than the avergae error rate in rankALS. The precision of SVD++ is 9 times lower than the one of RankaLS. Aditionally, SVD has high values of rmse and mae in comarison to the other two algorithms.

# Conclusion
RankALS outperforms SVD++. RankALS is better in terms of error rate, rmse and mae if significance weights are used, which supports the findings of the paper, but is worse in terms of recall and precision, which disproves the paper.

# Sources


1.   F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets:
History and Context. ACM Transactions on Interactive Intelligent
Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages.
DOI=http://dx.doi.org/10.1145/2827872
2.   Gábor Takács and Domonkos Tikk. 2012. Alternating least squares for personalized ranking. In Proceedings of the sixth ACM conference on Recommender systems (RecSys '12). Association for Computing Machinery, New York, NY, USA, 83–90. DOI:https://doi.org/10.1145/2365952.2365972